In [1]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor,BitsAndBytesConfig
from qwen_vl_utils import process_vision_info
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device is :{device}")

/home/aman/ChatVideoQwen2Vl/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device is :cuda


# Load Model

In [2]:
model_name = "Qwen/Qwen2-VL-7B-Instruct-AWQ"
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,torch_dtype=torch.bfloat16,attn_implementation="flash_attention_2"
).to(device)
processor = AutoProcessor.from_pretrained(model_name)

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


ValueError: You current version of `autoawq` does not support module quantization skipping, please upgrade `autoawq` package to at least 0.1.8.

# Chat Template

In [4]:
file_path='/home/aman/ChatVideoQwen2Vl/download/videoplayback.mp4'
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": f"{file_path}",
                "max_pixels": 360 * 420,
                "fps": 1.0,
            },
            {"type": "text", "text": "What is this Person Explaining about?."},
        ],
    }
]

messages

[{'role': 'user',
  'content': [{'type': 'video',
    'video': '/home/aman/ChatVideoQwen2Vl/download/videoplayback.mp4',
    'max_pixels': 151200,
    'fps': 1.0},
   {'type': 'text', 'text': 'What is this Person Explaining about?.'}]}]

# Apply Preprocessor Chat Template

In [5]:
text=processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
text

'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|video_pad|><|vision_end|>What is this Person Explaining about?.<|im_end|>\n<|im_start|>assistant\n'

In [6]:
image_inputs,video_inputs=process_vision_info(messages)
print(f"Image Inputs are :{image_inputs}")
print(f"Video Inputs are :{video_inputs}")
inputs=processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
).to(device)
print(f"Inputs are :{inputs}")

: 

# Generate Outputs

In [ ]:
generated_ids=model.generate(**inputs,max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

# Download Sample Video

In [19]:
from pytube import YouTube
import os

def create_downloadsdirectory():
    download_path = 'download'
    if not os.path.exists(download_path):
        os.makedirs(download_path)
    return download_path

def download_youtubevideo(url):
    download_path = create_downloadsdirectory()
    try:
        yt = YouTube(url)
        print(f"Title: {yt.title}")
        video_stream = yt.streams.get_highest_resolution()
        print(f"Downloading {yt.title}...")
        video_stream.download(output_path=download_path)
        print(f"Download complete! Video saved in: {download_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    video_url = 'https://www.youtube.com/watch?v=qqN63hbziaI'
    download_youtubevideo(video_url)

Title: What is LLM quantization?
An error occurred: HTTP Error 400: Bad Request
